In [1]:
from Resources.lib import *
from Resources import lib as lib
from Resources import model_package as MODEL

2024-02-09 17:42:06,577 numexpr.utils INFO Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-02-09 17:42:06,577 numexpr.utils INFO NumExpr defaulting to 8 threads.
2024-02-09 17:42:07.817906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 17:42:07.817952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 17:42:07.818883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 17:42:07.826338: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instruction

In [2]:
###----------------
### Some parametersimport warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
import tensorflow as tf
###----------------

inpDir = './upload'
outDir = '../DATA'
subDir = 'csvs'
audDir = "AUDIO"

RANDOM_STATE = 24               # REMEMBER: to remove at the time of promotion to production
np.random.seed(RANDOM_STATE)    # Set Random Seed for reproducible  results

EPOCHS = 11                     # number of epochs
ALPHA = 0.001                   # learning rate
NUM_SAMPLES = 1280              # How many samples we want to generate 
NOISE = 0.2                     # Noise to be introduced in the data
TEST_SIZE = 0.2
BATCH_SIZE = 26
PATIENCE = 20

# parameters for Matplotlib
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 8),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'
         }

CMAP = 'coolwarm' # plt.cm.Spectral

plt.rcParams.update(params)

In [3]:
df = pd.read_csv("./DATA/csvs/trump-original_REAL.csv")
df = df.drop(columns="LABEL")
df

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0.358431,0.047544,3542.994033,2472.263393,6142.101496,0.229714,-183.50685,34.215317,-25.610510,5.232159,...,-8.369213,-14.175573,-15.062232,-2.411484,-2.136527,4.616875,-3.939896,-5.642972,-8.739244,-9.369484
1,0.398888,0.013980,2956.441093,2525.691815,5812.986062,0.171032,-350.51126,58.881863,-23.915743,13.723481,...,-9.138926,-9.554494,-8.560787,-2.228990,-14.121047,-0.228751,-2.585347,-3.479855,-10.985689,-4.435715
2,0.277066,0.028064,2216.672834,2266.495699,4373.197798,0.103405,-328.82242,84.158720,-20.430117,6.658457,...,-4.212953,-9.122508,-6.528575,5.203862,-6.371897,-2.242783,-7.658451,-6.257457,-6.273143,-3.551254
3,0.359222,0.035303,2968.641140,2467.412969,5483.625932,0.178067,-265.03745,61.225735,-20.567955,-0.498796,...,-6.021339,-6.477277,-14.361066,0.192268,-5.031093,-0.998947,-3.826711,-2.932528,-9.419311,-3.316403
4,0.329339,0.024873,2111.408670,2129.420379,3848.325972,0.105125,-283.91092,91.656950,-41.411020,-11.166985,...,-9.240157,-2.310187,-10.009682,-3.353667,-7.186926,-0.003199,-9.301209,-5.654606,-5.261727,2.498810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.380453,0.032703,2666.514180,2112.847586,4668.300559,0.163064,-289.70065,79.301820,-34.634937,-2.610724,...,0.092124,1.182775,-15.351231,-2.042419,-9.185846,-2.628170,-11.762791,-5.393684,-8.502969,-2.474540
597,0.317955,0.036571,2310.894483,2100.396248,4078.339733,0.125388,-264.99075,87.606865,-33.935280,10.865050,...,4.615997,-5.449326,-13.275034,5.052628,-9.755420,-6.369123,-11.137697,-3.693965,-7.176722,-5.510276
598,0.417653,0.041490,1943.153338,1824.786318,3402.001398,0.105247,-205.10083,105.617240,-65.244415,-11.780135,...,-6.529221,-1.176161,-11.554144,-4.132590,-11.246901,0.278846,-11.095168,-3.507075,-4.943163,3.291278
599,0.401598,0.037857,2042.405086,1803.789276,3407.874090,0.118852,-200.52167,85.288734,-90.497860,-33.224705,...,1.590815,4.300089,-3.402193,-4.930651,-13.453516,-5.114317,-7.686624,1.444214,-2.637488,2.451039


In [7]:
df = lib.create_DataFrame("DATA/KAGGLE/AUDIO/FAKE/linus-to-trump.wav" , segment_length=1)

model = MODEL.Model()
model.buid_model()
model.load_weights("Weights/demo_weigths")
result = model.predict(df)

2024-02-09 17:42:42,892 LIB INFO Feature Extraction STARTED
2024-02-09 17:42:58,149 LIB INFO Feature Extraction SUCCESSFUL
2024-02-09 17:42:58,158 LIB INFO DataFrame creation SUCCESSFUL
2024-02-09 17:42:58,170 tensorflow WARNING Layer gru_2 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [8]:
result

' 98.25 Fake%'

In [6]:
lib.TF_Predict(df)

2024-02-09 17:42:14,500 tensorflow WARNING Layer gru_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


' 89.68 Fake%'

In [5]:
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_file, wav_file):
    try:
        # Load the MP3 file
        audio = AudioSegment.from_mp3(mp3_file)
        
        # Export the audio to WAV
        audio.export(wav_file, format="wav")
        
        print(f"MP3 file '{mp3_file}' converted to WAV file '{wav_file}' successfully.")
    except Exception as e:
        print(f"Error converting MP3 to WAV: {e}")


In [9]:
# Example usage:
convert_mp3_to_wav("DATA/DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3", "Upload/linus-original-DEMO.wav")

MP3 file 'DATA/DEMONSTRATION/DEMONSTRATION/linus-original-DEMO.mp3' converted to WAV file 'Upload/linus-original-DEMO.wav' successfully.


In [2]:
# !pip install pydub

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
file = "linus-original-DEMO.mp3"

In [12]:
"Upload/"+file[:-4] + ".wav"

'Upload/linus-original-DEMO.wav'